# [197. Rising Temperature](https://leetcode.com/problems/rising-temperature/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Weather

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| recordDate    | date    |
| temperature   | int     |
+---------------+---------+</pre>
id is the column with unique values for this table.
This table contains information about the temperature on a certain day.
 

Write a solution to find all dates' Id with higher temperatures compared to its previous dates (yesterday).

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Weather table:
<pre>+----+------------+-------------+
| id | recordDate | temperature |
+----+------------+-------------+
| 1  | 2015-01-01 | 10          |
| 2  | 2015-01-02 | 25          |
| 3  | 2015-01-03 | 20          |
| 4  | 2015-01-04 | 30          |
+----+------------+-------------+</pre>
Output: 
<pre>+----+
| id |
+----+
| 2  |
| 4  |
+----+</pre>
Explanation: 
In 2015-01-02, the temperature was higher than the previous day (10 -> 25).
In 2015-01-04, the temperature was higher than the previous day (20 -> 30).

In [0]:
#Pandas schema

import pandas as pd

data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype({'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'})

In [0]:
# to pyspark schema

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

weather_df = spark.createDataFrame(weather)
weather_df.show(truncate=False)

+---+-------------------+-----------+
|id |recordDate         |temperature|
+---+-------------------+-----------+
|1  |2015-01-01 00:00:00|10         |
|2  |2015-01-02 00:00:00|25         |
|3  |2015-01-03 00:00:00|20         |
|4  |2015-01-04 00:00:00|30         |
+---+-------------------+-----------+



In [0]:
(weather_df.alias("w1")
 .join(weather_df.alias("w2"), F.datediff(F.col("w1.recordDate"), F.col("w2.recordDate")) == 1, "inner")
 .where("w1.temperature > w2.temperature")
 .select("w1.id").show())

+---+
| id|
+---+
|  2|
|  4|
+---+



In [0]:
# in Spark SQL
weather_df.createOrReplaceTempView('weather')
spark.sql('''
select
    w1.id
from
    weather w1
    join weather w2 on datediff(w1.recordDate, w2.recordDate) = 1
where
    w1.temperature > w2.temperature
''').show()

+---+
| id|
+---+
|  2|
|  4|
+---+



In [0]:
spark.stop()